In [14]:
// Load IP2Location sym file and apply unique attribute
load`$"<REDACTED>";
ipSym:`u#ipSym;
//Load IP2Location table
load`$"<REDACTED>";
// Memory map IP2Location table
`<REDACTED> set -23!latest;

In [15]:
// Load AMDB sym file and apply unique attribute
load`$"<REDACTED>";
amdbSym:`u#<REDACTED>;
//Load AMDB table
load`$"<REDACTED>";
// Memory map accountRooftop table
`<REDACTED> set -23!<REDACTED>;

In [10]:
//Get Sample data
`sample_DMCIPListening set get hsym `$"/sharedfolder/cbath/Udacity_Bath/sample_DMCIPListening"
`sample_displayClicks set get hsym `$"/sharedfolder/cbath/Udacity_Bath/sample_displayClicks"

`sample_DMCIPListening


`sample_displayClicks


iabCat| x       
------| --------
IAB_19| 16251350
IAB_12| 15813876
IAB_1 | 6667752 
IAB_9 | 5306342 
IAB_13| 4317539 
IAB_5 | 3435528 
IAB_17| 3381793 
IAB_2 | 3086864 
IAB_22| 2609367 
IAB_18| 2603529 
IAB_3 | 1504153 
IAB_15| 1502963 
IAB_7 | 1477806 
IAB_10| 1345270 
IAB_21| 952165  
IAB_24| 900513  
IAB_4 | 705917  
IAB_26| 632545  
IAB_14| 553715  
IAB_20| 323956  
IAB_8 | 322736  
IAB_6 | 213837  
IAB_11| 101936  
IAB_23| 76871   
IAB_16| 39003   
IAB_25| 501     


In [50]:
desc update percentage:100*category_hits % sum category_hits from  select category_hits:sum hits by iabCat from sample_DMCIPListening
select count i by dates from sample_DMCIPListening
meta sample_DMCIPListening
desc select count i by domain from sample_DMCIPListening where iabCat = `IAB_22

iabCat| category_hits percentage  
------| --------------------------
IAB_19| 54438930      22.8086     
IAB_12| 46720538      19.57478    
IAB_22| 21940571      9.192569    
IAB_13| 17058480      7.147091    
IAB_2 | 15699047      6.577521    
IAB_9 | 15554319      6.516884    
IAB_15| 12930970      5.417764    
IAB_1 | 12834545      5.377364    
IAB_5 | 10042655      4.20763     
IAB_17| 5476575       2.294553    
IAB_18| 4632997       1.941114    
IAB_4 | 4564783       1.912534    
IAB_3 | 3421448       1.433504    
IAB_7 | 3062883       1.283274    
IAB_21| 2329386       0.9759564   
IAB_10| 2268393       0.9504018   
IAB_24| 1795450       0.7522501   
IAB_26| 1340668       0.5617075   
IAB_14| 1021625       0.4280362   
IAB_20| 498133        0.2087057   
IAB_8 | 412716        0.172918    
IAB_6 | 326080        0.1366196   
IAB_11| 166057        0.06957387  
IAB_23| 89640         0.03755699  
IAB_16| 49724         0.02083315  
IAB_25| 643           0.0002694015


dates     | x      
----------| -------
2019.03.01| 2822119
2019.03.02| 1028807
2019.03.03| 912680 
2019.03.04| 2984996
2019.03.05| 3194964
2019.03.06| 3198318
2019.03.07| 3051411
2019.03.08| 2801804
2019.03.09| 940048 
2019.03.10| 907217 
2019.03.11| 3133880
2019.03.12| 3556608
2019.03.13| 3563747
2019.03.14| 3336628
2019.03.15| 2892899
2019.03.16| 1005541
2019.03.17| 921702 
2019.03.18| 3286674
2019.03.19| 3430769
2019.03.20| 3093637
2019.03.21| 3014266
2019.03.22| 2717976
2019.03.23| 954542 
2019.03.24| 882644 
2019.03.25| 3100017
2019.03.26| 3152064
2019.03.27| 3100503
2019.03.28| 2886345
2019.03.29| 2647127
2019.03.30| 831684 
2019.03.31| 776210 


c          | t f a
-----------| -----
dates      | d    
domains    | s    
company    | s    
userDomain | s    
city       | s    
region     | s    
countryCode| s    
iabCat     | s    
hits       | j    
pageViews  | j    
uniqueViews| j    


[0;31mdomain[0m: [0;31mdomain[0m

In [12]:
//Funks
.reports.util.addMetricByLabelValueColumns:{[table;byCols;metricCol;labelCol] .debug.addPivotValueColumns.input:`table`byCols`metricCol`labelCol!(table;byCols;metricCol;labelCol);
    //Clean Inputs 
    table: 0!table;
    /table: @[0!table;exec c from meta[table] where t="s";value]; /Remove enumeration and keys
    byCols: distinct except[(),byCols;`];
    metricCol: first (),metricCol;
    labelCol: first (),labelCol;
    //Function to create column for metrics by label condition
    deriveMetric:{[metric;label;labelVal] (enlist `$string[metric],"_",string[label],"_",string `unknown^labelVal)!enlist(sum;(metric;(&:;(=;label;enlist labelVal))))};
    sCl: raze deriveMetric[metricCol;labelCol;] each distinct table[labelCol];
    //Return table with new columns
    :?[0!table;();byCols!byCols;sCl];
 };

.reports.util.getMetricsByGroup:{[table;countCols;sumCols;groupCols].debug.getMetricsByGroup.input:`table`countCols`sumCols`groupCols!(table;countCols;sumCols;groupCols);
    //Enforce consistant list type
    countCols,:();sumCols,:();groupCols,:();
    //Remove nulls
    countCols:countCols except `;sumCols:sumCols except `;groupCols:groupCols except `;
    //Sanity checks for necessary details - groupCol can all be null
    metricCols:countCols,sumCols;	
    if[(not count metricCols) or (count missing:(metricCols,groupCols) where not (metricCols,groupCols) in `i,cols table);
        msg:".reports.util.getMetricsByGroup - Unusable input parameters.Columns missing - ",raze string missing;
        .log.warn[.z.h;msg;()];
        'msg
    ];
    //By clause setup - enforced list type, so same treatment for atomic and list of group columns
    byClause:$[count groupCols;(groupCols)!(groupCols);0b];	
    //Count (distinct) statement setup - change to generic metric names
    countStatement:$[count countCols;raze{(enlist `$string[x],"Count")!(enlist(count;(distinct;x)))}each countCols;()];
    //Sum statement setup - - change to generic metric names
    sumStatement:$[count sumCols;raze{(enlist `$string[x],"Sum")!(enlist(sum;x))}each sumCols;()];
    //Create Metrics table
    resultsTab:?[table;();byClause;countStatement,sumStatement];	
    //Results schema must be consistant - TEMP HACK FOR KX DASHBOARD NOT REFRESHING EMPTY PIE CHARTS
    resultsTab:$[count resultsTab;resultsTab;1#resultsTab];
    //TODO - HACK FOR DASHBOARDS AS CANNOT HANDLE LONGS
    :@[0!resultsTab;exec c from meta resultsTab where t in "jl";"f"$];
 };

.reports.util.cleanText:{[table]
    //kdb+ does not think to quote-escape symbols with commas, but it will if they're strings
    table:@[table; exec c from meta[table]where t="s";string];
    //kdb+ does not think to quote-escape anything with line break characters, we have to do that ourselves
    // but it *is* smart enough to quote-escape strings with quotes in them (go figure)
    escapeNewLines: {?["\n" in/: x; "\"",/:x,'"\""; x]};
    table:@[table; exec c from meta[table]where t="C";escapeNewLines];
    :table;
 }

In [13]:
//Build Dataset
base: sample_DMCIPListening;
display: sample_displayClicks
//Attach "account" level metrics
ipdbAccountCols:`company`userDomain`city`region`countryCode;
//Hits by iabCat
baseX1:.reports.util.addMetricByLabelValueColumns[base;ipdbAccountCols;`hits;`iabCat];
//All other metrics
baseX2: 0!.reports.util.getMetricsByGroup[base;`dates`domains;`hits`pageViews`uniqueViews;ipdbAccountCols];
//Combine all data
k:`company`userDomain`city`region;
baseX:(lj/)(k xkey baseX2;k xkey `company`userDomain xcol `org`domain xcols display;k xkey baseX1);

In [16]:
//Attach all account level labels and extra data
baseX: select from ij[baseX; 4!select company:org, userDomain:domain,city,region,acct_id,hq_id,revenue_mil_usd,total_employees,naic3,naic6 from 0!<REDACTED>] where 0<count each hq_id;
baseX: baseX lj 4!select company:org,userDomain:domain,city,region,isISP,usageType from <REDACTED>;

In [42]:
//Tidy up data
accountCols:`acct_id`hq_id`company`userDomain`city`region`countryCode`revenue_mil_usd`total_employees`naic3`naic6`isISP`usageType;
//Nice IAB Order
a:cols[baseX] where cols[baseX] like "hits_*";
iabCols:key asc a!"I"$("IAB_" vs/: string a)[;1];
metricCols:distinct `datesCount`domainsCount`hitsSum`pageViewsSum`uniqueViewsSum`clicks`clickDates,iabCols;
final: accountCols xcols 0!baseX;
final: @[final;metricCols;0^];
final: (accountCols,metricCols) xcols .reports.util.cleanText[final];

//q Table (FLat and dirty)
(hsym `$"/sharedfolder/cbath/Udacity_Bath/sample_model_data") set final;

//CSV
(hsym `$"/sharedfolder/cbath/Udacity_Bath/sample_model_data.csv") 0: "," 0: final;

count final
meta final


283329


c                 | t f a
------------------| -----
acct_id           | C    
hq_id             | C    
company           | C    
userDomain        | C    
city              | C    
region            | C    
countryCode       | C    
revenue_mil_usd   | f    
total_employees   | j    
naic3             | h    
naic6             | i    
isISP             | b    
usageType         | C    
datesCount        | f    
domainsCount      | f    
hitsSum           | f    
pageViewsSum      | f    
uniqueViewsSum    | f    
clicks            | j    
clickDates        | j    
hits_iabCat_IAB_1 | j    
hits_iabCat_IAB_2 | j    
hits_iabCat_IAB_3 | j    
hits_iabCat_IAB_4 | j    
hits_iabCat_IAB_5 | j    
hits_iabCat_IAB_6 | j    
hits_iabCat_IAB_7 | j    
hits_iabCat_IAB_8 | j    
hits_iabCat_IAB_9 | j    
hits_iabCat_IAB_10| j    
hits_iabCat_IAB_11| j    
hits_iabCat_IAB_12| j    
hits_iabCat_IAB_13| j    
hits_iabCat_IAB_14| j    
hits_iabCat_IAB_15| j    
hits_iabCat_IAB_16| j    
hits_iabCat_